### Import

In [146]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, GlobalAveragePooling1D
from sklearn.utils import class_weight
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt

### Organize the data

In [148]:
# load in the npz
data = np.load('tremor_data.npz')

# Assuming keys are 'features' and 'labels'
X = data["features"]    # shape should be (3091, 300, 3)
y = data["labels"]      # shape should be (3091,)
print(X.shape, y.shape)

unique, counts = np.unique(y, return_counts=True)
print(dict(zip(unique, counts)))

(3091, 300, 3) (3091,)
{0: 2997, 1: 28, 2: 66}


### Train/test split

In [150]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

### One-hot encode labels

In [152]:
y_train_cat = to_categorical(y_train, num_classes=3)
y_test_cat = to_categorical(y_test, num_classes=3)

### Architecture for 1D CNN

In [164]:

def build_model(input_length, num_channels=3, num_classes=3):
    """
    input_length: number of time steps in each sample
    num_channels: 3 for accelerometer (x,y,z)
    num_classes: 3 for pre-tremor (0)
    """
    model = Sequential([
        # 1st conv layer
        # arbitrary --> 32 filters of size 3 to input sequence
        # relu activation (introduces non-linearity)
        # input_shape --> specifies shape of each sample (time steps, channels)
        Conv1D(filters=32, kernel_size=3, activation='relu', padding='same', input_shape=(input_length, num_channels)),
        
        # 1st max pooling layer
        # reduce sequence length by taking max value in each window of size 2 (arbitrary)
        # reduces computation + extracts dominant features
        MaxPooling1D(pool_size=2),

        # 2nd conv layer
        # 64 filters of size 3
        Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'),
        
        # 2nd max ppling layer
        MaxPooling1D(pool_size=2),

        # flatten the 3d output (time steps, features) into 1D vector
        # necessary to connect conv layers to fully connected (dense) layers
        # Flatten(),
        GlobalAveragePooling1D(),

        # fully connected layer
        # 64 neurons, relu activation
        Dense(64, activation='relu'),

        # dropout layer --> regularization
        # randomly set 50% of input to 0 during training to prevent overfitting
        Dropout(0.5),

        # output layer
        # single neuron with sigmoid activation (binary classification)
        Dense(num_classes, activation='softmax')
    ])

    # compile with binary cross-entropy for tremor vs no tremor
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

# example usage (say there are 200 time steps per sample)
model = build_model(input_length=200, num_channels=3)
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_26 (Conv1D)              │ (None, 200, 32)        │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_22 (MaxPooling1D) │ (None, 100, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_27 (Conv1D)              │ (None, 100, 64)        │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_23 (MaxPooling1D) │ (None, 50, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_6      │ (None, 64)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,883 (42.51 KB)

 Trainable params: 10,883 (42.51 KB)

 Non-trainable params: 0 (0.00 B)

### Compute weights

In [166]:
# weights = class_weight.compute_class_weight(
#     class_weight='balanced',
#     classes=np.unique(y_train),
#     y=y_train
# )
# class_weights = dict(enumerate(weights))
# print(class_weights)

### Training the model

In [168]:
history = model.fit(
    X_train, y_train_cat,
    validation_split = 0.3,
    epochs = 20,
    batch_size = 32,
    verbose = 1
)

Epoch 1/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8821 - loss: 203.3616 - val_accuracy: 0.9584 - val_loss: 127.5955
Epoch 2/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8838 - loss: 46.6999 - val_accuracy: 0.9492 - val_loss: 23.4811
Epoch 3/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9040 - loss: 12.0342 - val_accuracy: 0.9561 - val_loss: 5.9792
Epoch 4/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9029 - loss: 2.7246 - val_accuracy: 0.9538 - val_loss: 3.5192
Epoch 5/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9630 - loss: 1.2797 - val_accuracy: 0.9561 - val_loss: 0.9554
Epoch 6/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9561 - loss: 0.9900 - val_accuracy: 0.9515 - val_loss: 0.7016
Epoch 7/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9618 - loss: 0.7963 - val_accuracy: 0.9538 - val_loss: 0.6557
Epoch 8/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9723 - loss: 0.6075 - val_accuracy: 0.9584 - va

### Evaluate the model

In [170]:
test_loss, test_acc = model.evaluate(X_test, y_test_cat, verbose=0)
print(f"Test accuracy: {test_acc:.4f}")

Test accuracy: 0.9709


### Predictions & confusion matrix

In [178]:
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)
print("\nClassification Report:\n", classification_report(y_test, y_pred))

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Confusion Matrix:
 [[899   0   1]
 [  8   0   0]
 [ 18   0   2]]

Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.99       900
           1       0.00      0.00      0.00         8
           2       0.67      0.10      0.17        20

    accuracy                           0.97       928
   macro avg       0.55      0.37      0.39       928
weighted avg       0.96      0.97      0.96       928



/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
